In [ ]:
import sys
sys.path.insert(0, "..")
import pickle
from gpbp_osm.layers import AdmArea
from gpbp_osm import visualisation
from gpbp_osm.distance import calculate_isopolygons_graph, calculate_isopolygons_Mapbox, population_served

import osmnx as ox
import warnings
warnings.filterwarnings(action='ignore')

In [2]:
adm_area = AdmArea(country="Timor-Leste", level=0)

Retrieving data for Timor-Leste of granularity level 0
Extracting geometry for Timor-Leste


In [ ]:
adm_area.get_facilities(method="osm", tags={"building":"hospital"})
visualisation.plot_facilities(adm_area.fac_gdf)

In [4]:
adm_area.get_population(method="world_pop")
#visualisation.plot_population_heatmap(adm_area.pop_df)

Data downloaded
Converting raster file to dataframe


In [ ]:
adm_area.compute_potential_fac(spacing=0.4)
visualisation.plot_facilities(adm_area.grid_gdf)

In [ ]:
pop_count, current, potential = adm_area.prepare_optimization_data("length", [2000, 5000, 10000], "driving", "mapbox", population_resolution=2)

In [ ]:
from gpbp_osm.utils import group_population
from gpbp_osm.visualisation import plot_population
pop_grouped = group_population(adm_area.pop_df, nof_digits=2)
plot_population(pop_grouped)

In [20]:
from optimization import jg_opt
from functools import partial
# Complains that current is dict -> convert to DataFrame
cplex_optimize = partial(jg_opt.OpenOptimize, solver="cplex")
jg_opt.Solve(pop_count, current, potential, "length", [5, 10, 20], optimize=jg_opt.Optimize, type='ID')

(       10000      5000      2000
 5   0.024952  0.009178  0.001889
 10  0.025205  0.009254  0.001917
 20  0.025205  0.009254  0.001917,
                            10000                      5000  \
 5           [10, 13, 16, 14, 11]      [10, 13, 16, 12, 11]   
 10  [10, 13, 16, 14, 11, 12, 15]  [10, 13, 16, 12, 11, 14]   
 20  [10, 13, 16, 14, 11, 12, 15]  [10, 13, 16, 12, 11, 14]   
 
                         2000  
 5       [10, 16, 13, 14, 11]  
 10  [10, 16, 13, 14, 11, 12]  
 20  [10, 16, 13, 14, 11, 12]  )